# Title


### Class def

In [ ]:
import pyvisa
import time

class RigolInstrument:
    def __init__(self, resource_name):
        self.inst = pyvisa.ResourceManager('@py').open_resource(resource_name)
        self.inst.timeout = 5000
        self.inst.encoding = 'utf-8'
        self.idn = self.get_idn()
        self.instrument_type = self.classify_instrument()

    def get_idn(self):
        try:
            self.inst.write('*IDN?')
            time.sleep(0.1)
            return self.inst.read().strip()
        except pyvisa.errors.VisaIOError:
            return "Unknown"

    def classify_instrument(self):
        idn_upper = self.idn.upper()
        if "RIGOL" in idn_upper and "DG" in idn_upper:
            return "generator"
        elif "RIGOL" in idn_upper and ("DS" in idn_upper or "MSO" in idn_upper):
            return "oscilloscope"
        else:
            return "unknown"

    def write(self, command):
        self.inst.write(command)

    def read(self):
        return self.inst.read()

    def close(self):
        self.inst.close()
    
    def __str__(self):
        return f"{self.instrument_type.capitalize()} - IDN: {self.idn}"

    def toggle(channel, state, self):
        # ------ CH1 ----------
        if channel == "CH1":
            if state == "ON":
                self.write("OUTP ON")
            else:
                self.write("OUTP OFF")
        # ------ CH2 ----------
        elif channel == "CH2":
            if state == "ON":
                self.write('OUTPut:CH2 ON ')
            else:
                self.write('OUTPut:CH2 OFF ')
        return

    def generate_waveform(slef, channel, waveform_type, frequency, amplitude, offset=0, phase=0, duty_cycle=50):
        """Note that writing to CH1 is different from writing to CH2...."""
        args = f"{frequency}, {amplitude}, {offset} "
        
        if channel == "CH1":
            # print(channel)
            self.write('VOLT:UNIT VPP')
            time.sleep(0.05)
            self.write(f'APPL:{waveform_type} {args}')   # e.g. APPL:RAMP 1000, 2.5, 0.5
            time.sleep(0.05)
            self.write(f'PHAS {phase} ')
            time.sleep(0.05)
            toggle("CH1", "ON", dg)

        else:
            # for CH2 use the CH-specific notation...
            self.write(f'VOLT:UNIT:CH{ch} VPP')
            time.sleep(0.05)
            self.write(f'APPL:{waveform_type}:CH{ch} {args}')  # e.g. APPL:SQUare:CH2 1000, 2.5, 0.5
            time.sleep(0.05)
            self.write(f'PHAS:CH{ch} {phase} ')
            time.sleep(0.05)
            toggle(f"CH{ch}", "ON", self)



def detect_rigol_instruments():
    """Retourne un tuple (generator, oscilloscope) si présents, sinon None."""
    rm = pyvisa.ResourceManager('@py')
    devices = [d for d in rm.list_resources() if "USB" in d]

    generator = None
    oscilloscope = None
    print("Detecting Rigol instruments...")

    for dev in devices:
        rigol = RigolInstrument(dev)
        if rigol.instrument_type == "generator":
            generator = rigol
            print(f"Detected Generator: {generator}")
        elif rigol.instrument_type == "oscilloscope":
            oscilloscope = rigol
            print(f"Detected Oscilloscope: {oscilloscope}")

    return generator, oscilloscope


generator, oscilloscope = detect_rigol_instruments()

if generator:
    generator.close()

if oscilloscope:
    oscilloscope.close()

    

Detecting Rigol instruments...


AttributeError: 'NoneType' object has no attribute 'close'

### Rigol DG1022 :


In [ ]:


# 1 - Connect to DG1022Z (and DS1054Z)
generator, oscilloscope = detect_rigol_instruments()

# 2 - Toggle CH1, CH2
generator.write("OUTP1 ON")
time.sleep(5)
generator.write("OUTP1 OFF")
time.sleep(1)
generator.write("OUTP2 ON")
time.sleep(5)
generator.write("OUTP2 OFF")
time.sleep(1)

# 3 - Sine Function
wave = {
    "sin":"SINusoid",
    "square" : "SQUare",
    "ramp" : "RAMP",
    "pulse" : "PULSe",
    "noise" : "NOISe",
    "dc" : "DC"
}

for w in wave:
    generator.generate_waveform(channel="CH1", waveform_type=w, frequency=1000, amplitude=5)
    print(f'CH1 is a : {w}')
    time.sleep(2)

for w in wave:
    generator.generate_waveform(channel="CH2", waveform_type=w, frequency=1000, amplitude=5)
    print(f'CH2 is a : {w}')
    time.sleep(2)


generator.close()
oscilloscope.close()




Detecting Rigol instruments...
